In [1]:
import numpy as np
import pandas as pd
import Bio as bio
import re
import os
import glob
from Bio import SeqIO
import pprint
from BioSQL import BioSeqDatabase
from operator import add

In [2]:
i = 0
gb_file = r'C:\Users\Mole\Documents\Uni work\UNI WORK\Year 4\The project\Data\Human_RNA\human_4_rna.gbff'
mega_file = SeqIO.parse(open(gb_file,"r"), "genbank")
for record in mega_file:
    i+=1
    if i % 100 == 0:
        print(i)
print(i)


KeyboardInterrupt: 

In [3]:
record.features

[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(3375), strand=1), type='source'),
 SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(3375), strand=1), type='gene'),
 SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(1102), strand=1), type='exon'),
 SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(1), strand=1), type='variation'),
 SeqFeature(FeatureLocation(ExactPosition(8), ExactPosition(16), strand=1), type='variation'),
 SeqFeature(FeatureLocation(ExactPosition(11), ExactPosition(12), strand=1), type='variation'),
 SeqFeature(FeatureLocation(ExactPosition(16), ExactPosition(17), strand=1), type='variation'),
 SeqFeature(FeatureLocation(ExactPosition(19), ExactPosition(20), strand=1), type='variation'),
 SeqFeature(FeatureLocation(ExactPosition(20), ExactPosition(21), strand=1), type='variation'),
 SeqFeature(FeatureLocation(ExactPosition(22), ExactPosition(23), strand=1), type='variation'),
 SeqFeature(FeatureLocation(ExactPosition(23), ExactP

In [4]:
mega_file

In [5]:
record

SeqRecord(seq=Seq('GCGGAGGCTTGGGGCAGCCGGGTAGCTCGGAGGTCGTGGCGCTGGGGGCTAGCA...AAA'), id='NM_001171622.2', name='NM_001171622', description='Homo sapiens vascular endothelial growth factor A (VEGFA), transcript variant 8, mRNA', dbxrefs=[])

In [6]:
record.features[8].type

'variation'

In [7]:
i = 0
while i < len(record.features):
    if record.features[i].type == 'CDS':
        print(i)
        print('cds')
        i+= 1
    i+=1

127
cds


In [8]:
len(record.features)

994

# Code for extracting gene

In [9]:
x = record.features[8]

y = str(x)
z = re.search('location:(.+?)]',y)
lo = z.group(1)
lo = lo[lo.find('['):]
if ' ' in lo:
    lo = lo.replace(' ','')
if '[' in lo:
    loc = lo.replace('[','')
mid = loc.find(':')
loc1 = int(loc[:mid])
loc2 = int(loc[mid+1:])
gene = str(record.seq[loc1:loc2])
print(record.features[1].type)
print(gene)
print(type(gene))

gene
G
<class 'str'>


In [10]:
GC_per = (gene.count('G')+gene.count('C'))/len(gene)
GC_per

1.0

# Function gene extraction returns gene

In [11]:
def code_rev(data,j):
    x = data.features[j]
    y = str(x)
    z = re.search('location:(.+?)]',y)
    lo = z.group(1)
    loc = lo[lo.find('['):]
    if ' ' in loc:
        loc = loc.replace(' ','')
    if '[' in loc:
        loc = loc.replace('[','')
    if '{' in loc:
        loc = loc.replace('{','')
    mid = loc.find(':')
    loc1 = int(loc[:mid])
    loc2 = int(loc[mid+1:])
    gene = str(data.seq[loc1:loc2])
    if data.features[j].strand == -1:
        gene = gene[::-1]
        i = 0
        gene_rev = ''
        while i < len(gene):
            if gene[i] =='A':
                gene_rev +='T'
            elif gene[i] == 'T':
                gene_rev += 'A'
            elif gene[i] == 'G':
                gene_rev += 'C'
            elif gene[i] =='C':
                gene_rev += 'G'
            i+=1
        del(gene)
        gene = gene_rev
    return(gene)


In [12]:
code_rev(record,10)

'A'

# Codon finder

In [13]:
j = 0
while j < len(record.features):
    if record.features[j].type == 'CDS':
        codon = code_rev(record,j)
        print(codon[-3:])
        print(j)
        j+= 1
    j+=1

    

TGA
127


In [14]:
stop_codon = []
i= 0
for record in data:
    print(i)
    if i <10:
        while j < len(record.features):
            if record.features[j].type == 'CDS':
                codon = code_rev(record,j)
                codon_3 = codon[-3:]
                stop_codon.append(codon_3)
                j+= 1
                i+=1
            else:
                j+=1
                i+=1
    else:
        print(stop_codon)
        break
        

NameError: name 'data' is not defined

In [15]:
data

NameError: name 'data' is not defined

In [16]:
for record in data:
    print(record.seq)

NameError: name 'data' is not defined

In [17]:
data = record

In [18]:
j = 0
cds_no = 0
codon_data = [['TAA',0],['TAG',0],['TGA',0],['unk',0]]
codon_GC_data = [['TAA'],['TAG'],['TGA']]
codon_GC3_data = [['TAA'],['TAG'],['TGA']]
codon_unk =[]
codon4_unk = []
global_gene = code_rev(data,0)
global_GC = (global_gene.count('G')+global_gene.count('C'))/len(global_gene)
global_GC3 = GC_3(global_gene)
codon_four = [['TAAA',0],['TAAT',0],['TAAG',0],['TAAC',0],['TGAA',0],['TGAT',0],['TGAG',0],['TGAC',0],['TAGA',0],['TAGT',0],['TAGG',0],['TAGC',0],['unk',0]]
while j < len(data.features):
    try:
        if data.features[j].type == 'CDS':
            cds_no +=1
            gene = code_rev(data,j)
            codon = gene[-3:]
            GC_per = (gene.count('G')+gene.count('C'))/len(gene)
            GC_3per = GC_3(gene)
            if codon == 'TAA':
                codon_data[0][1] = codon_data[0][1]+1
                codon_GC_data[0].append(GC_per)
                codon_GC3_data[0].append(GC_3per)
                codon_four = four_stop(data,j,codon_four)
                j+=1
            elif codon == 'TAG':
                codon_data[1][1] = codon_data[1][1]+1
                codon_GC_data[1].append(GC_per)
                codon_GC3_data[1].append(GC_3per)
                codon_four = four_stop(data,j,codon_four)
                j+=1
            elif codon == 'TGA':
                codon_data[2][1] = codon_data[2][1]+1
                codon_GC_data[2].append(GC_per)
                codon_GC3_data[2].append(GC_3per)
                codon_four = four_stop(data,j,codon_four)
                j+=1
            else:
                codon_data[3][1] = codon_data[3][1]+1
                codon_four = four_stop(data,j,codon_four)
                codon_unk.append(codon)
                j+=1
        else:
            j+=1
    except:
        j+=1
codon_df = pd.DataFrame(codon_GC_data, index = ['TAA','TAG','TGA'])
print(codon_df.mean(axis=1))   
codon_GC3df = pd.DataFrame(codon_GC3_data, index = ['TAA','TAG','TGA'])
print(codon_GC3df.mean(axis=1))

print(cds_no - (codon_data[0][1]+codon_data[1][1]+codon_data[2][1]))
print(codon_data)
print(global_GC)
print(codon_four)
print(global_GC3)
print(data.description,codon_unk)


NameError: name 'GC_3' is not defined

In [19]:
def GC_3(gene):
    i=0
    stop = []
    GC3_NA = []
    while i+3 <= int((len(gene)/3)):
        if gene[(i*3+3)]== 'A': 
            GC3_NA.append('A')
            i +=1
        elif gene[(i*3+3)]== 'T': 
            GC3_NA.append('T')
            i+=1
        elif gene[(i*3+3)]== 'G': 
            GC3_NA.append('G')
            i+=1
        elif gene[(i*3+3)]== 'C': 
            GC3_NA.append('C')
            i+=1
        else:
            i+=1
    GC3 = 0
    GC3 = ((GC3_NA.count('G') + GC3_NA.count('C'))/(len(gene)/3))    
    return(GC3)

In [20]:
def four_stop(data,j,codon_four):
    x = data.features[j]
    y = str(x)
    z = re.search('location:(.+?)]',y)
    lo = z.group(1)
    loc = lo[lo.find('['):]
    if ' ' in loc:
        loc = loc.replace(' ','')
    if '[' in loc:
        loc = loc.replace('[','')
    if '{' in loc:
        loc = loc.replace('{','')
    mid = loc.find(':')
    if data.features[j].strand ==1: 
        loc1 = int(loc[:mid])
        loc2 = int((loc[mid+1:]))+1
        gene = str(data.seq[loc1:loc2])
    elif data.features[j].strand == -1:
        loc1 = int(loc[:mid])-1
        loc2 = int((loc[mid+1:]))
        gene = str(data.seq[loc1:loc2])
        gene = gene[::-1]
        i = 0
        gene_rev = ''
        while i < len(gene):
            if gene[i] =='A':
                gene_rev +='T'
            elif gene[i] == 'T':
                gene_rev += 'A'
            elif gene[i] == 'G':
                gene_rev += 'C'
            elif gene[i] =='C':
                gene_rev += 'G'
            i+=1
        del(gene)
        gene = gene_rev
    stopper = gene[-4:]
    ar = np.array(codon_four)
    u = np.where(ar== stopper)
    try:
        p= [u[0]]
        q = int(p[0])
        codon_four[q][1] = codon_four[q][1]+1
    except:
        codon_four[12][1] = codon_four[12][1]+1
    return(codon_four)

In [21]:
def file_runner(data):
    j = 0
    codon_data = [['TAA',0],['TAG',0],['TGA',0],['unk',0]]
    codon_unk =[]
    codon_GC_data = [['TAA'],['TAG'],['TGA']]
    codon_GC3_data = [['TAA'],['TAG'],['TGA']]
    global_gene = code_rev(data,0)
    global_GC = (global_gene.count('G')+global_gene.count('C'))/len(global_gene)
    global_GC3 = GC_3(global_gene)
    codon_four = [['TAAA',0],['TAAT',0],['TAAG',0],['TAAC',0],['TGAA',0],['TGAT',0],['TGAG',0],['TGAC',0],['TAGA',0],['TAGT',0],['TAGG',0],['TAGC',0],['unk',0]]
    while j < len(data.features):
        try:
            if data.features[j].type == 'CDS':
                gene = code_rev(data,j)
                codon = gene[-3:]
                GC_per = (gene.count('G')+gene.count('C'))/len(gene)
                GC_3per = GC_3(gene)
                if codon == 'TAA':
                    codon_data[0][1] = codon_data[0][1]+1
                    codon_GC_data[0].append(GC_per)
                    codon_GC3_data[0].append(GC_3per)
                    codon_four = four_stop(data,j,codon_four)
                    j+=1
                elif codon == 'TAG':
                    codon_data[1][1] = codon_data[1][1]+1
                    codon_GC_data[1].append(GC_per)
                    codon_GC3_data[1].append(GC_3per)
                    codon_four = four_stop(data,j,codon_four)
                    j+=1
                elif codon == 'TGA':
                    codon_data[2][1] = codon_data[2][1]+1
                    codon_GC_data[2].append(GC_per)
                    codon_GC3_data[2].append(GC_3per)
                    codon_four = four_stop(data,j,codon_four)
                    j+=1
                else:
                    codon_data[3][1] = codon_data[3][1]+1
                    codon_four = four_stop(data,j,codon_four)
                    j+=1
                    if len(codon_unk) ==0:
                        codon_unk.append(data.description)
                        codon_unk.append(codon)
                    else:
                        codon_unk.append(codon)
                    
            else:
                j+=1
        except:
            j+=1
    codon_df = pd.DataFrame(codon_GC_data, index = ['TAA','TAG','TGA'])
    codon_GC3df = pd.DataFrame(codon_GC3_data, index = ['TAA','TAG','TGA'])
    GC_list = (codon_df.mean(axis = 1))
    GC3_list = (codon_GC3df.mean(axis=1))
    Name_list = ['Name','TAA','TGA','TAG','Global_GC','Global_GC3','TAA_GC','TGA_GC','TAG_GC','TAA_GC3','TGA_GC3','TAG_GC3','TAAA','TAAT','TAAG','TAAC','TGAA','TGAT','TGAG','TGAC','TAGA','TAGT','TAGG','TAGC','Unk_Stop','Unk_four']
    file_data = [data.description,codon_data[0][1],codon_data[2][1],codon_data[1][1],(global_GC),(global_GC3),GC_list[0],GC_list[2],GC_list[1],GC3_list[0],GC3_list[2],GC3_list[1],codon_four[0][1],codon_four[1][1],codon_four[2][1],codon_four[3][1],codon_four[4][1],codon_four[5][1],codon_four[6][1],codon_four[7][1],codon_four[8][1],codon_four[9][1],codon_four[10][1],codon_four[11][1],codon_data[3][1],codon_four[12][1]]
    df_file_data = pd.DataFrame([file_data],columns = Name_list)
    df_file_data = df_file_data.set_index('Name')
    return([df_file_data,codon_unk])
    


In [22]:
file_runner(record)

[                                                    TAA  TGA  TAG  Global_GC  \
 Name                                                                           
 Homo sapiens vascular endothelial growth factor...    0    1    0   0.515556   
 
                                                     Global_GC3  TAA_GC  \
 Name                                                                     
 Homo sapiens vascular endothelial growth factor...    0.567111     NaN   
 
                                                       TGA_GC  TAG_GC  TAA_GC3  \
 Name                                                                            
 Homo sapiens vascular endothelial growth factor...  0.661426     NaN      NaN   
 
                                                     TGA_GC3  ...  TGAA  TGAT  \
 Name                                                         ...               
 Homo sapiens vascular endothelial growth factor...  0.68239  ...     0     0   
 
                                   

In [23]:
Name_list = ['Name','TAA','TGA','TAG','Global_GC','Global_GC3','TAA_GC','TGA_GC','TAG_GC','TAA_GC3','TGA_GC3','TAG_GC3','TAAA','TAAT','TAAG','TAAC','TGAA','TGAT','TGAG','TGAC','TAGA','TAGT','TAGG','TAGC','Unk_Stop','Unk_four']

In [24]:
archea_df = pd.DataFrame(columns = Name_list)
archea_df = archea_df.set_index('Name')
archea_df

,TAA,TGA,TAG,Global_GC,Global_GC3,TAA_GC,TGA_GC,TAG_GC,TAA_GC3,TGA_GC3,...,TGAA,TGAT,TGAG,TGAC,TAGA,TAGT,TAGG,TAGC,Unk_Stop,Unk_four
Name,,,,,,,,,,,,,,,,,,,,,


In [25]:
archea_df = pd.DataFrame(columns = Name_list)
archea_df = archea_df.set_index('Name')
holder_2 = file_runner(data)
full_archea_df = holder_2[0]
unknown_stops = pd.DataFrame(holder_2[1])
full_archea_df

,TAA,TGA,TAG,Global_GC,Global_GC3,TAA_GC,TGA_GC,TAG_GC,TAA_GC3,TGA_GC3,...,TGAA,TGAT,TGAG,TGAC,TAGA,TAGT,TAGG,TAGC,Unk_Stop,Unk_four
Name,,,,,,,,,,,,,,,,,,,,,
"Homo sapiens vascular endothelial growth factor A (VEGFA), transcript variant 8, mRNA",0,1,0,0.515556,0.567111,NaN,0.661426,NaN,NaN,0.68239,...,0,0,1,0,0,0,0,0,0,0


# Adjusted Runner each gene seperate


In [251]:
i = 0
unk_stops = []
gb_file = r'C:\Users\Mole\Documents\Uni work\UNI WORK\Year 4\The project\Data\Human_RNA\human_1_rna.gbff'
mega_file = SeqIO.parse(open(gb_file,"r"), "genbank")
for record in mega_file:
    if i <10:
        holder = file_runner(record)
        archea_df = archea_df.append(holder[0])
        if holder[1] != []:
            unk_stops.append(holder[1])
        i+=1
    else:
        break
        
        

In [214]:
archea_df

,TAA,TGA,TAG,Global_GC,Global_GC3,TAA_GC,TGA_GC,TAG_GC,TAA_GC3,TGA_GC3,...,TGAA,TGAT,TGAG,TGAC,TAGA,TAGT,TAGG,TAGC,Unk_Stop,Unk_four
Name,,,,,,,,,,,,,,,,,,,,,
"Homo sapiens collagen type XIII alpha 1 chain (COL13A1), transcript variant 26, mRNA",0,1,0,0.584505,0.563593,NaN,0.643661,NaN,NaN,0.773006,...,0,1,0,0,0,0,0,0,0,0
"Homo sapiens collagen type XIII alpha 1 chain (COL13A1), transcript variant 27, mRNA",0,1,0,0.588255,0.521654,NaN,0.633194,NaN,NaN,0.788703,...,0,1,0,0,0,0,0,0,0,0
"Homo sapiens transforming acidic coiled-coil containing protein 1 (TACC1), transcript variant 32, non-coding RNA",0,0,0,0.360846,0.368810,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
"Homo sapiens transforming acidic coiled-coil containing protein 1 (TACC1), transcript variant 38, non-coding RNA",0,0,0,0.360967,0.369171,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
"Homo sapiens SWI/SNF related, matrix associated, actin dependent regulator of chromatin, subfamily a, member 4 (SMARCA4), transcript variant 8, mRNA",0,1,0,0.604384,0.427945,NaN,0.611558,NaN,NaN,0.632198,...,0,0,1,0,0,0,0,0,0,0
"Homo sapiens transforming acidic coiled-coil containing protein 1 (TACC1), transcript variant 37, non-coding RNA",0,0,0,0.373073,0.364379,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
"Homo sapiens small integral membrane protein 35 (SMIM35), mRNA",0,1,0,0.566416,0.571429,NaN,0.557471,NaN,NaN,0.568966,...,0,1,0,0,0,0,0,0,0,0
"Homo sapiens post-GPI attachment to proteins 2 (PGAP2), transcript variant 22, non-coding RNA",0,0,0,0.543628,0.487759,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
"Homo sapiens Rho guanine nucleotide exchange factor 10 like (ARHGEF10L), transcript variant 7, non-coding RNA",0,0,0,0.622620,0.486672,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [26]:
archea_df = pd.DataFrame(columns = Name_list)
archea_df = archea_df.set_index('Name')
archea_df

,TAA,TGA,TAG,Global_GC,Global_GC3,TAA_GC,TGA_GC,TAG_GC,TAA_GC3,TGA_GC3,...,TGAA,TGAT,TGAG,TGAC,TAGA,TAGT,TAGG,TAGC,Unk_Stop,Unk_four
Name,,,,,,,,,,,,,,,,,,,,,


In [27]:
i = 0
unk_stops = []
gb_file = r'C:\Users\Mole\Documents\Uni work\UNI WORK\Year 4\The project\Data\Human_RNA\human_8_rna.gbff'
mega_file = SeqIO.parse(open(gb_file,"r"), "genbank")
for record in mega_file:
    if i <10000000:
        holder = file_runner(record)
        archea_df = archea_df.append(holder[0])
        if holder[1] != []:
            unk_stops.append(holder[1])
        if i % 100 == 0:
            print(i)
        i+=1
    else:
        break
            
    



0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [277]:
i = 0
gb_file = r'C:\Users\Mole\Documents\Uni work\UNI WORK\Year 4\The project\Data\Human_RNA\human_1_rna.gbff'
mega_file = SeqIO.parse(open(gb_file,"r"), "genbank")
for record in mega_file:
    while i < 10:
        while j < len(record.features):
            if record.features[j].type == 'CDS':
                print(record.features[j])
                j+=1
            else:
                j+=1
        i+=1
    else:
        break

In [263]:
mega_file

In [28]:
archea_df

,TAA,TGA,TAG,Global_GC,Global_GC3,TAA_GC,TGA_GC,TAG_GC,TAA_GC3,TGA_GC3,...,TGAA,TGAT,TGAG,TGAC,TAGA,TAGT,TAGG,TAGC,Unk_Stop,Unk_four
Name,,,,,,,,,,,,,,,,,,,,,
"PREDICTED: Homo sapiens leucine rich repeats and guanylate kinase domain containing (LRGUK), transcript variant X9, mRNA",0,1,0,0.436075,0.450216,NaN,0.437835,NaN,NaN,0.519293,...,0,0,0,1,0,0,0,0,0,0
"PREDICTED: Homo sapiens leucine rich repeats and guanylate kinase domain containing (LRGUK), transcript variant X10, misc_RNA",0,0,0,0.436788,0.469430,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
"PREDICTED: Homo sapiens leucine rich repeats and guanylate kinase domain containing (LRGUK), transcript variant X11, misc_RNA",0,0,0,0.445758,0.479692,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
"PREDICTED: Homo sapiens leucine rich repeats and guanylate kinase domain containing (LRGUK), transcript variant X12, misc_RNA",0,0,0,0.438733,0.471963,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
"PREDICTED: Homo sapiens scavenger receptor cysteine rich family member with 4 domains (SSC4D), transcript variant X1, mRNA",0,1,0,0.626819,0.659044,NaN,0.663361,NaN,NaN,0.720661,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Homo sapiens STX5 divergent transcript (STX5-DT), long non-coding RNA",0,0,0,0.557522,0.584071,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
"Homo sapiens AQP5 and AQP2 antisense RNA 2 (AQP5-AS1), transcript variant 1, long non-coding RNA",0,0,0,0.590219,0.632378,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
"Homo sapiens PRMT5 divergent transcript (PRMT5-DT), long non-coding RNA",0,0,0,0.413669,0.410072,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [112]:
unk_stops

[]

In [29]:
archea_df['sum'] = archea_df['TAA'] + archea_df['TGA'] + archea_df['TAG']
indexNames = archea_df[ archea_df['sum'] == 0 ].index
archea_df.drop(indexNames , inplace=True)

archea_df

,TAA,TGA,TAG,Global_GC,Global_GC3,TAA_GC,TGA_GC,TAG_GC,TAA_GC3,TGA_GC3,...,TGAT,TGAG,TGAC,TAGA,TAGT,TAGG,TAGC,Unk_Stop,Unk_four,sum
Name,,,,,,,,,,,,,,,,,,,,,
"PREDICTED: Homo sapiens leucine rich repeats and guanylate kinase domain containing (LRGUK), transcript variant X9, mRNA",0,1,0,0.436075,0.450216,NaN,0.437835,NaN,NaN,0.519293,...,0,0,1,0,0,0,0,0,0,1
"PREDICTED: Homo sapiens scavenger receptor cysteine rich family member with 4 domains (SSC4D), transcript variant X1, mRNA",0,1,0,0.626819,0.659044,NaN,0.663361,NaN,NaN,0.720661,...,0,0,1,0,0,0,0,0,0,1
"PREDICTED: Homo sapiens scavenger receptor cysteine rich family member with 4 domains (SSC4D), transcript variant X2, mRNA",0,1,0,0.625048,0.580982,NaN,0.679771,NaN,NaN,0.729730,...,0,0,1,0,0,0,0,0,0,1
"PREDICTED: Homo sapiens chromosome 7 open reading frame 31 (C7orf31), transcript variant X3, mRNA",0,1,0,0.365942,0.346618,NaN,0.407699,NaN,NaN,0.488189,...,0,1,0,0,0,0,0,0,0,1
"PREDICTED: Homo sapiens corticotropin releasing hormone receptor 2 (CRHR2), transcript variant X1, mRNA",0,1,0,0.499774,0.463690,NaN,0.571197,NaN,NaN,0.507282,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Homo sapiens caspase recruitment domain family member 8 (CARD8), transcript variant 18, mRNA",0,0,1,0.407085,0.415215,NaN,NaN,0.516204,NaN,NaN,...,0,0,0,0,0,0,1,0,0,1
"Homo sapiens heterogeneous nuclear ribonucleoprotein U (HNRNPU), transcript variant 1, mRNA",0,1,0,0.418087,0.386632,NaN,0.503632,NaN,NaN,0.607748,...,0,0,0,0,0,0,0,0,0,1
"Homo sapiens tumor protein p53 (TP53), transcript variant 2, mRNA",0,1,0,0.533679,0.569151,NaN,0.572770,NaN,NaN,0.585915,...,0,0,1,0,0,0,0,0,0,1


In [30]:
archea_df.to_csv(r'C:\Users\Mole\Documents\Uni work\UNI WORK\Year 4\The project\Extracted_csvs\human_8_gene_csv')

In [70]:
def file_runner(data):
    j = 0
    codon_data = [['TAA',0],['TAG',0],['TGA',0],['unk',0]]
    codon_unk =[]
    codon_GC_data = [['TAA',0],['TAG',0],['TGA',0]]
    codon_GC3_data = [['TAA',0],['TAG',0],['TGA',0]]
    global_gene = code_rev(data,0)
    global_GC = (global_gene.count('G')+global_gene.count('C'))/len(global_gene)
    global_GC3 = GC_3(global_gene)
    codon_four = [['TAAA',0],['TAAT',0],['TAAG',0],['TAAC',0],['TGAA',0],['TGAT',0],['TGAG',0],['TGAC',0],['TAGA',0],['TAGT',0],['TAGG',0],['TAGC',0],['unk',0]]
    while j < len(data.features):
        try:
            if data.features[j].type == 'CDS':
                gene = code_rev(data,j)
                codon = gene[-3:]
                GC_per = (gene.count('G')+gene.count('C'))/len(gene)
                GC_3per = GC_3(gene)
                if codon == 'TAA':
                    codon_data[0][1] = codon_data[0][1]+1
                    del(codon_GC_data[0][1])
                    codon_GC_data[0].append(GC_per)
                    del(codon_GC3_data[0][1])
                    codon_GC3_data[0].append(GC_3per)
                    codon_four = four_stop(data,j,codon_four)
                    j+=1
                elif codon == 'TAG':
                    codon_data[1][1] = codon_data[1][1]+1
                    del(codon_GC_data[1][1])
                    codon_GC_data[1].append(GC_per)
                    del(codon_GC3_data[1][1])
                    codon_GC3_data[1].append(GC_3per)
                    codon_four = four_stop(data,j,codon_four)
                    j+=1
                elif codon == 'TGA':
                    codon_data[2][1] = codon_data[2][1]+1
                    del(codon_GC_data[2][1])
                    codon_GC_data[2].append(GC_per)
                    del(codon_GC3_data[2][1])
                    codon_GC3_data[2].append(GC_3per)
                    codon_four = four_stop(data,j,codon_four)
                    j+=1
                else:
                    codon_data[3][1] = codon_data[3][1]+1
                    codon_four = four_stop(data,j,codon_four)
                    j+=1
                    if len(codon_unk) ==0:
                        codon_unk.append(data.description)
                        codon_unk.append(codon)
                    else:
                        codon_unk.append(codon)
                    
            else:
                j+=1
        except:
            j+=1
    codon_df = pd.DataFrame(codon_GC_data, index = ['TAA','TAG','TGA'])
    codon_GC3df = pd.DataFrame(codon_GC3_data, index = ['TAA','TAG','TGA'])
    GC_list = (codon_df.mean(axis = 1))
    GC3_list = (codon_GC3df.mean(axis=1))
    Name_list_GC = ['Global_GC','TAA','TGA','TAG','TAAA','TAAT','TAAG','TAAC','TGAA','TGAT','TGAG','TGAC','TAGA','TAGT','TAGG','TAGC','Unk_Stop','Unk_four']
    Name_list_GC3 = ['Global_GC3','TAA','TGA','TAG','TAAA','TAAT','TAAG','TAAC','TGAA','TGAT','TGAG','TGAC','TAGA','TAGT','TAGG','TAGC','Unk_Stop','Unk_four']
    file_data_GC = [(global_GC*100),codon_data[0][1],codon_data[2][1],codon_data[1][1],codon_four[0][1],codon_four[1][1],codon_four[2][1],codon_four[3][1],codon_four[4][1],codon_four[5][1],codon_four[6][1],codon_four[7][1],codon_four[8][1],codon_four[9][1],codon_four[10][1],codon_four[11][1],codon_data[3][1],codon_four[12][1]]
    file_data_GC3 = [(global_GC3*100),codon_data[0][1],codon_data[2][1],codon_data[1][1],codon_four[0][1],codon_four[1][1],codon_four[2][1],codon_four[3][1],codon_four[4][1],codon_four[5][1],codon_four[6][1],codon_four[7][1],codon_four[8][1],codon_four[9][1],codon_four[10][1],codon_four[11][1],codon_data[3][1],codon_four[12][1]]
    return([file_data_GC,file_data_GC3])
    


In [46]:
holder = file_runner(record)
tester_data = holder[0]
tester_data


[43.6075036075036, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

In [47]:
Name_list_GC = ['Global_GC','TAA','TGA','TAG','TAAA','TAAT','TAAG','TAAC','TGAA','TGAT','TGAG','TGAC','TAGA','TAGT','TAGG','TAGC','Unk_Stop','Unk_four']
len(Name_list_GC)

18

In [48]:
Name_list = ['Global_GC','TAA','TGA','TAG','TAAA','TAAT','TAAG','TAAC','TGAA','TGAT','TGAG','TGAC','TAGA','TAGT','TAGG','TAGC','Unk_Stop','Unk_four']
Homo_GC_data = pd.DataFrame(columns = Name_list)
Homo_GC_data = Homo_GC_data.set_index('Global_GC')
Homo_GC_data


,TAA,TGA,TAG,TAAA,TAAT,TAAG,TAAC,TGAA,TGAT,TGAG,TGAC,TAGA,TAGT,TAGG,TAGC,Unk_Stop,Unk_four
Global_GC,,,,,,,,,,,,,,,,,


In [49]:
Name_list_GC3 = ['Global_GC3','TAA','TGA','TAG','TAAA','TAAT','TAAG','TAAC','TGAA','TGAT','TGAG','TGAC','TAGA','TAGT','TAGG','TAGC','Unk_Stop','Unk_four']


In [50]:
Homo_GC3_data = pd.DataFrame(columns = Name_list_GC3)
Homo_GC3_data = Homo_GC3_data.set_index('Global_GC3')
Homo_GC3_data

,TAA,TGA,TAG,TAAA,TAAT,TAAG,TAAC,TGAA,TGAT,TGAG,TGAC,TAGA,TAGT,TAGG,TAGC,Unk_Stop,Unk_four
Global_GC3,,,,,,,,,,,,,,,,,


In [51]:
def file_adder(GC_data,new_data):
    i = 0
    while i < 10:
        if new_data[0] < GC_data[i][0]:
            x = (list(map(add, GC_data[i][1:], new_data[1:])))
            x.insert(0,GC_full[i][0])
            del(GC_data[i])
            GC_data.insert(i,x)
            return(GC_data)
        else:
            i+=1


In [52]:
i = 0
unk_stops = []
GC_full = [[10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[70,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[80,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[90,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
GC3_full = [[10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[70,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[80,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[90,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
gb_file = r'C:\Users\Mole\Documents\Uni work\UNI WORK\Year 4\The project\Data\Human_RNA\human_8_rna.gbff'
mega_file = SeqIO.parse(open(gb_file,"r"), "genbank")
for record in mega_file:
    if i <100:
        holder = file_runner(record)
        fresh_data = holder[0]
        GC_full = file_adder(GC_full,fresh_data)
        fre3sh_data = holder[1]
        GC3_full = file_adder(GC3_full,fre3sh_data)
        i+=1
    else:
        z= 0
        while z < len(GC_full):
            df_file_data = pd.DataFrame([GC_full[z]],columns = Name_list)
            df_file_data = df_file_data.set_index('Global_GC')
            Homo_GC_data = Homo_GC_data.append(df_file_data)
                        
            df_file_data3 = pd.DataFrame([GC3_full[z]], columns = Name_list_GC3)
            df_file_data3 = df_file_data3.set_index('Global_GC3')
            Homo_GC3_data = Homo_GC3_data.append(df_file_data3)
            
            z+=1
        break
z= 0
while z < len(GC_full):
    df_file_data = pd.DataFrame([GC_full[z]],columns = Name_list)
    df_file_data = df_file_data.set_index('Global_GC')
    Homo_GC_data = Homo_GC_data.append(df_file_data)

    df_file_data3 = pd.DataFrame([GC3_full[z]], columns = Name_list_GC3)
    df_file_data3 = df_file_data3.set_index('Global_GC3')
    Homo_GC3_data = Homo_GC3_data.append(df_file_data3)

    z+=1

In [53]:
Homo_GC_data

,TAA,TGA,TAG,TAAA,TAAT,TAAG,TAAC,TGAA,TGAT,TGAG,TGAC,TAGA,TAGT,TAGG,TAGC,Unk_Stop,Unk_four
Global_GC,,,,,,,,,,,,,,,,,
10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40,4,16,4,1,0,2,1,13,1,2,0,0,0,3,1,0,0
50,16,27,7,0,3,1,12,16,1,6,4,2,0,5,0,0,0
60,0,7,0,0,0,0,0,0,2,5,0,0,0,0,0,0,0
70,0,3,1,0,0,0,0,0,0,1,2,1,0,0,0,0,0
80,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
90,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [54]:
Homo_GC3_data

,TAA,TGA,TAG,TAAA,TAAT,TAAG,TAAC,TGAA,TGAT,TGAG,TGAC,TAGA,TAGT,TAGG,TAGC,Unk_Stop,Unk_four
Global_GC3,,,,,,,,,,,,,,,,,
10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40,14,19,7,1,1,2,10,13,1,5,0,2,0,5,0,0,0
50,4,26,3,0,2,1,1,16,2,4,4,1,0,1,1,0,0
60,2,7,2,0,0,0,2,0,1,5,1,0,0,2,0,0,0
70,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
80,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
90,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [87]:
Homo_GC_data.to_csv(r'C:\Users\Mole\Documents\Uni work\UNI WORK\Year 4\The project\Extracted_csvs\human_8_GC_csv')

In [88]:
Homo_GC3_data.to_csv(r'C:\Users\Mole\Documents\Uni work\UNI WORK\Year 4\The project\Extracted_csvs\human_8_GC3_csv')

In [121]:
GC_full = [['1-10',0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[8,8,8,8,8]]
eee = [9,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
len(eee)

25

In [221]:
Name_list = ['GC','TAA','TGA','TAG','Global_GC','Global_GC3','TAA_GC','TGA_GC','TAG_GC','TAA_GC3','TGA_GC3','TAG_GC3','TAAA','TAAT','TAAG','TAAC','TGAA','TGAT','TGAG','TGAC','TAGA','TAGT','TAGG','TAGC','Unk_Stop','Unk_four']
Name_list3 = ['GC','TAA','TGA','TAG','Global_GC','Global_GC3','TAA_GC','TGA_GC','TAG_GC','TAA_GC3','TGA_GC3','TAG_GC3','TAAA','TAAT','TAAG','TAAC','TGAA','TGAT','TGAG','TGAC','TAGA','TAGT','TAGG','TAGC','Unk_Stop','Unk_four']


In [222]:
Name_list = ['GC','TAA','TGA','TAG','Global_GC','Global_GC3','TAA_GC','TGA_GC','TAG_GC','TAA_GC3','TGA_GC3','TAG_GC3','TAAA','TAAT','TAAG','TAAC','TGAA','TGAT','TGAG','TGAC','TAGA','TAGT','TAGG','TAGC','Unk_Stop','Unk_four']
archea_df = pd.DataFrame(columns = Name_list)
archea_df = archea_df.set_index('GC')
archea_df


,TAA,TGA,TAG,Global_GC,Global_GC3,TAA_GC,TGA_GC,TAG_GC,TAA_GC3,TGA_GC3,...,TGAA,TGAT,TGAG,TGAC,TAGA,TAGT,TAGG,TAGC,Unk_Stop,Unk_four
GC,,,,,,,,,,,,,,,,,,,,,


In [225]:
Name_list3 = ['AA','TAA','TGA','TAG','Global_GC','Global_GC3','TAA_GC','TGA_GC','TAG_GC','TAA_GC3','TGA_GC3','TAG_GC3','TAAA','TAAT','TAAG','TAAC','TGAA','TGAT','TGAG','TGAC','TAGA','TAGT','TAGG','TAGC','Unk_Stop','Unk_four']
archea_df3 = pd.DataFrame(columns = Name_list)
archea_df3 = archea_df.set_index('GC')
archea_df3

KeyError: "None of ['GC'] are in the columns"

In [262]:
GC_full = [[10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[70,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[80,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[90,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
x = file_adder(GC_full,tester_data)
print(x)
len(x[0])

[[10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [40, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [50, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [60, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [70, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [80, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


19

In [264]:
Name_list_GC = ['Global_GC','TAA','TGA','TAG','TAAA','TAAT','TAAG','TAAC','TGAA','TGAT','TGAG','TGAC','TAGA','TAGT','TAGG','TAGC','Unk_Stop','Unk_four']
len(Name_list_GC)
file_data_GC = [(global_GC*100),codon_data[2][1],codon_data[1][1],codon_four[0][1],codon_four[1][1],codon_four[2][1],codon_four[3][1],codon_four[4][1],codon_four[5][1],codon_four[6][1],codon_four[7][1],codon_four[8][1],codon_four[9][1],codon_four[10][1],codon_four[11][1],codon_data[3][1],codon_four[12][1]]
len(file_data_GC)

17

In [238]:
x = (list( map(add, GC_full[0][1:], eee[1:]) ))
x.insert(0,GC_full[0][0])
print(x)
x = divmod(x[0],2)
x

[10, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


(5, 0)

In [177]:
x = [[1,2,3,4],[1,2,3,4]]
x.insert(1,[10,10,10])
x

[[1, 2, 3, 4], [10, 10, 10], [1, 2, 3, 4]]

In [241]:
myArray = np.array([10,20,30,40,50,60,70,80,90])
myInt = 10
newArray = myArray myArray[0]/myInt
newArray

1.0

In [280]:
GC_full = [[10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[70,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[80,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[90,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
len(GC_full[2])

19

In [356]:
df = [[0,0,0,1,2,3],[1,0,0,4,5,6,],[0,0,1,5,6,7]]

In [357]:
ee = pd.DataFrame(df)
ee

,0,1,2,3,4,5
0,0,0,0,1,2,3
1,1,0,0,4,5,6
2,0,0,1,5,6,7


In [358]:
ee['e'] = ee[0] + ee[1] + ee[2]
ee

,0,1,2,3,4,5,e
0,0,0,0,1,2,3,0
1,1,0,0,4,5,6,1
2,0,0,1,5,6,7,1


In [359]:
ee
x = sum(ee[0] + ee[1] + ee[2])
print(x)


2


In [360]:
indexNames = ee[ ee['e'] == 0 ].index
ee.drop(indexNames , inplace=True)

ee

,0,1,2,3,4,5,e
1,1,0,0,4,5,6,1
2,0,0,1,5,6,7,1


In [328]:
archea_df

,TAA,TGA,TAG,Global_GC,Global_GC3,TAA_GC,TGA_GC,TAG_GC,TAA_GC3,TGA_GC3,...,TGAA,TGAT,TGAG,TGAC,TAGA,TAGT,TAGG,TAGC,Unk_Stop,Unk_four
Name,,,,,,,,,,,,,,,,,,,,,
"Homo sapiens collagen type XIII alpha 1 chain (COL13A1), transcript variant 26, mRNA",0,1,0,0.584505,0.563593,NaN,0.643661,NaN,NaN,0.773006,...,0,1,0,0,0,0,0,0,0,0
"Homo sapiens collagen type XIII alpha 1 chain (COL13A1), transcript variant 27, mRNA",0,1,0,0.588255,0.521654,NaN,0.633194,NaN,NaN,0.788703,...,0,1,0,0,0,0,0,0,0,0
"Homo sapiens transforming acidic coiled-coil containing protein 1 (TACC1), transcript variant 32, non-coding RNA",0,0,0,0.360846,0.368810,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
"Homo sapiens transforming acidic coiled-coil containing protein 1 (TACC1), transcript variant 38, non-coding RNA",0,0,0,0.360967,0.369171,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
"Homo sapiens SWI/SNF related, matrix associated, actin dependent regulator of chromatin, subfamily a, member 4 (SMARCA4), transcript variant 8, mRNA",0,1,0,0.604384,0.427945,NaN,0.611558,NaN,NaN,0.632198,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Homo sapiens methyl-CpG binding protein 2 (MECP2), transcript variant 5, mRNA",0,1,0,0.521316,0.523490,NaN,0.604061,NaN,NaN,0.588832,...,0,0,0,1,0,0,0,0,0,0
"Homo sapiens coagulation factor VIII (F8), transcript variant 2, mRNA",0,1,0,0.424694,0.457569,NaN,0.465438,NaN,NaN,0.488479,...,0,0,1,0,0,0,0,0,0,0
"Homo sapiens potassium channel tetramerization domain containing 7 (KCTD7), transcript variant 1, mRNA",0,1,0,0.501335,0.528651,NaN,0.601149,NaN,NaN,0.648276,...,0,0,1,0,0,0,0,0,0,0


In [362]:
archea_df['sum'] = archea_df['TAA'] + archea_df['TGA'] + archea_df['TAG']
archea_df

,TAA,TGA,TAG,Global_GC,Global_GC3,TAA_GC,TGA_GC,TAG_GC,TAA_GC3,TGA_GC3,...,TGAT,TGAG,TGAC,TAGA,TAGT,TAGG,TAGC,Unk_Stop,Unk_four,sum
Name,,,,,,,,,,,,,,,,,,,,,
"Homo sapiens collagen type XIII alpha 1 chain (COL13A1), transcript variant 26, mRNA",0,1,0,0.584505,0.563593,NaN,0.643661,NaN,NaN,0.773006,...,1,0,0,0,0,0,0,0,0,1
"Homo sapiens collagen type XIII alpha 1 chain (COL13A1), transcript variant 27, mRNA",0,1,0,0.588255,0.521654,NaN,0.633194,NaN,NaN,0.788703,...,1,0,0,0,0,0,0,0,0,1
"Homo sapiens transforming acidic coiled-coil containing protein 1 (TACC1), transcript variant 32, non-coding RNA",0,0,0,0.360846,0.368810,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
"Homo sapiens transforming acidic coiled-coil containing protein 1 (TACC1), transcript variant 38, non-coding RNA",0,0,0,0.360967,0.369171,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
"Homo sapiens SWI/SNF related, matrix associated, actin dependent regulator of chromatin, subfamily a, member 4 (SMARCA4), transcript variant 8, mRNA",0,1,0,0.604384,0.427945,NaN,0.611558,NaN,NaN,0.632198,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Homo sapiens methyl-CpG binding protein 2 (MECP2), transcript variant 5, mRNA",0,1,0,0.521316,0.523490,NaN,0.604061,NaN,NaN,0.588832,...,0,0,1,0,0,0,0,0,0,1
"Homo sapiens coagulation factor VIII (F8), transcript variant 2, mRNA",0,1,0,0.424694,0.457569,NaN,0.465438,NaN,NaN,0.488479,...,0,1,0,0,0,0,0,0,0,1
"Homo sapiens potassium channel tetramerization domain containing 7 (KCTD7), transcript variant 1, mRNA",0,1,0,0.501335,0.528651,NaN,0.601149,NaN,NaN,0.648276,...,0,1,0,0,0,0,0,0,0,1


In [363]:
indexNames = archea_df[ archea_df['sum'] == 0 ].index
archea_df.drop(indexNames , inplace=True)

archea_df

,TAA,TGA,TAG,Global_GC,Global_GC3,TAA_GC,TGA_GC,TAG_GC,TAA_GC3,TGA_GC3,...,TGAT,TGAG,TGAC,TAGA,TAGT,TAGG,TAGC,Unk_Stop,Unk_four,sum
Name,,,,,,,,,,,,,,,,,,,,,
"Homo sapiens collagen type XIII alpha 1 chain (COL13A1), transcript variant 26, mRNA",0,1,0,0.584505,0.563593,NaN,0.643661,NaN,NaN,0.773006,...,1,0,0,0,0,0,0,0,0,1
"Homo sapiens collagen type XIII alpha 1 chain (COL13A1), transcript variant 27, mRNA",0,1,0,0.588255,0.521654,NaN,0.633194,NaN,NaN,0.788703,...,1,0,0,0,0,0,0,0,0,1
"Homo sapiens SWI/SNF related, matrix associated, actin dependent regulator of chromatin, subfamily a, member 4 (SMARCA4), transcript variant 8, mRNA",0,1,0,0.604384,0.427945,NaN,0.611558,NaN,NaN,0.632198,...,0,1,0,0,0,0,0,0,0,1
"Homo sapiens small integral membrane protein 35 (SMIM35), mRNA",0,1,0,0.566416,0.571429,NaN,0.557471,NaN,NaN,0.568966,...,1,0,0,0,0,0,0,0,0,1
"Homo sapiens collagen type XIII alpha 1 chain (COL13A1), transcript variant 26, mRNA",0,1,0,0.584505,0.563593,NaN,0.643661,NaN,NaN,0.773006,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Homo sapiens methyl-CpG binding protein 2 (MECP2), transcript variant 5, mRNA",0,1,0,0.521316,0.523490,NaN,0.604061,NaN,NaN,0.588832,...,0,0,1,0,0,0,0,0,0,1
"Homo sapiens coagulation factor VIII (F8), transcript variant 2, mRNA",0,1,0,0.424694,0.457569,NaN,0.465438,NaN,NaN,0.488479,...,0,1,0,0,0,0,0,0,0,1
"Homo sapiens potassium channel tetramerization domain containing 7 (KCTD7), transcript variant 1, mRNA",0,1,0,0.501335,0.528651,NaN,0.601149,NaN,NaN,0.648276,...,0,1,0,0,0,0,0,0,0,1


In [ ]:
archea_df['sum'] = archea_df['TAA'] + archea_df['TGA'] + archea_df['TAG']
indexNames = archea_df[ archea_df['sum'] == 0 ].index
archea_df.drop(indexNames , inplace=True)

archea_df